In [17]:
import torch

In [18]:
def simcim_function(J, h=None, steps=1000, agents=1, dt=0.01, c0=0.9, sigma=0.03, a0=6.5):
    """
    Run a Simulated Coherent Ising Machine (SimCIM) simulation.
    
    Args:
        J: Coupling matrix defining the optimization problem
        h: External field vector (optional)
        steps: Number of simulation steps
        agents: Number of parallel optimization agents
        dt: Time step size
        c0: Momentum factor
        sigma: Noise standard deviation
        a0: Maximum pump parameter
    
    Returns:
        binary_solution: The final binary solution (±1 values)
        ising_energy: The Ising energy of the final solution
    """
    # Convert inputs to float tensors
    J = J.float()
    if h is not None:
        h = h.float()
    else:
        h = torch.zeros(J.shape[0], device=J.device)
    
    # Get problem size
    N = J.shape[0]
    device = J.device
    
    # Initialize positions and momenta
    x = torch.zeros((N, agents), dtype=torch.float32, device=device)  # Position variables
    y = torch.zeros_like(x)                                           # Momentum variables
    h = h.reshape(-1, 1)  # Make h a column vector
    
    # Define a(t) function - tanh schedule from negative to positive
    t_values = torch.linspace(-3, 3, steps, device=device)
    a_t_values = (torch.tanh(t_values) - 1) * a0
    
    # Initialize history arrays (if needed for debugging)
    # x_history = torch.zeros((steps + 1, N, agents), device=device)
    # x_history[0] = x.clone()
    
    # Main simulation loop
    for step, a_t in enumerate(a_t_values):
        # Calculate noise term
        noise = torch.randn(N, agents, device=device) * sigma
        
        # Calculate force term: a(t)*x + coupling forces + noise
        force = x * a_t + (torch.matmul(J, x) + h) * dt + noise
        
        # Update momenta with momentum conservation
        y = y * c0 + force * (1 - c0)
        
        # Apply updates to positions
        outside_range = torch.abs(x + y) >= 1.0
        x = x + y * (~outside_range)
        
        # Apply inelastic walls: for any |x_i| > 1
        x = torch.clamp(x, -1.0, 1.0)
        
        # Set corresponding momenta to 0 where walls were hit
        y[outside_range] = 0.0
        
        # Store current positions (if tracking history)
        # x_history[step+1] = x.clone()
    
    # Get binary solution by taking the sign
    binary_solution = torch.sign(x).squeeze(1)
    
    # Calculate Ising energy: -0.5 * ∑∑J_ij*s_i*s_j - ∑h_i*s_i
    ising_energy = -0.5 * torch.sum(binary_solution @ J * binary_solution, dim=0) - torch.matmul(h.T, binary_solution)
    
    return binary_solution, ising_energy.squeeze()

In [19]:
def sfc_function(J, h=None, steps=1000, agents=1, dt=0.1, k=0.2):
    """
    Run a Coherent Ising Machine with Separated Feedback Control (SFC) simulation.
    
    Args:
        J: Coupling matrix defining the optimization problem
        h: External field vector (optional)
        steps: Number of simulation steps
        agents: Number of parallel optimization agents
        dt: Time step size
        k: Parameter of deviation between mean-field and error variables
    
    Returns:
        binary_solution: The final binary solution (±1 values)
        ising_energy: The Ising energy of the final solution
    """
    # Convert inputs to float tensors
    J = J.float()
    if h is not None:
        h = h.float()
    else:
        h = torch.zeros(J.shape[0], device=J.device)
    
    # Get problem size
    N = J.shape[0]
    device = J.device
    
    # Initialize positions, momenta, and error variables
    x = torch.randn(N, agents, device=device) * 0.1   # Position variables
    e = torch.zeros((N, agents), dtype=torch.float32, device=device)  # Error variables
    h = h.reshape(-1, 1)  # Make h a column vector
    
    # Parameter schedules
    p_values = torch.linspace(-1, 1, steps, device=device)           # Pump parameter
    beta_values = torch.linspace(0.3, 0, steps, device=device)       # Error decay
    c_values = torch.linspace(1, 3, steps, device=device)            # Coupling strength
    
    # Calculate scaling factor for coupling
    xi = torch.sqrt(torch.tensor(2.0 * N / torch.sum(J**2), device=device))
    
    # Main simulation loop
    for step in range(steps):
        # Get current parameter values
        p_t = p_values[step]      # Current pump parameter
        beta_t = beta_values[step]  # Current error decay
        c_t = c_values[step]      # Current coupling strength
        
        # Calculate mean-field term (z)
        z = -xi * (torch.matmul(J, x) + h)
        
        # Apply error correction with tanh nonlinearity
        f = torch.tanh(c_t * z)
        
        # Update positions (cubic nonlinearity, pump, feedback, error correction)
        x = x + (-x**3 + (p_t - 1) * x - f - k * (z - e)) * dt
        
        # Update error variables
        e = e + (-beta_t * (e - z)) * dt
        
        # Apply boundaries (if needed)
        outside_range = torch.abs(x) > 1.0
        x = torch.clamp(x, -1.0, 1.0)
    
    # Get binary solution by taking the sign
    binary_solution = torch.sign(x).squeeze(1)
    
    # Calculate Ising energy: -0.5 * ∑∑J_ij*s_i*s_j - ∑h_i*s_i
    ising_energy = -0.5 * torch.sum(binary_solution @ J * binary_solution, dim=0) - torch.matmul(h.T, binary_solution)
    
    return binary_solution, ising_energy.squeeze()


In [20]:
def cfc_function(J, h=None, steps=1000, agents=1, dt=0.1):
    """
    Run a Coherent Ising Machine with Chaotic Feedback Control (CFC) simulation.
    
    Args:
        J: Coupling matrix defining the optimization problem
        h: External field vector (optional)
        steps: Number of simulation steps
        agents: Number of parallel optimization agents
        dt: Time step size
    
    Returns:
        binary_solution: The final binary solution (±1 values)
        ising_energy: The Ising energy of the final solution
    """
    # Convert inputs to float tensors
    J = J.float()
    if h is not None:
        h = h.float()
    else:
        h = torch.zeros(J.shape[0], device=J.device)
    
    # Get problem size
    N = J.shape[0]
    device = J.device
    
    # Initialize positions and error variables
    x = torch.randn(N, agents, device=device) * 0.1   # Position variables
    e = torch.ones_like(x)                           # Error variables
    h = h.reshape(-1, 1)  # Make h a column vector
    
    # Define parameter schedules
    rise_steps = int(0.9 * steps)
    plateau_steps = steps - rise_steps
    
    # Create pump parameter schedule (p_t)
    p_rise = torch.linspace(-1, 1, rise_steps, device=device)
    p_plateau = torch.ones(plateau_steps, device=device)
    p_values = torch.cat([p_rise, p_plateau])
    
    # Set fixed parameters
    alpha = 1.0  # Target amplitude
    beta = 0.15  # Error feedback gain
    
    # Calculate scaling factor for coupling
    xi = torch.sqrt(torch.tensor(2.0 * N / torch.sum(J**2), device=device))
    
    # Main simulation loop
    for step in range(steps):
        # Get current pump parameter
        p_t = p_values[step]
        
        # Calculate feedback term with error modulation
        z = xi * e * (torch.matmul(J, x) + h)
        
        # Update positions (cubic nonlinearity, pump, feedback)
        x = x + (-x**3 + (p_t - 1) * x + z) * dt
        
        # Update error variables (chaotic amplitude control)
        e = e + (-beta * e * (z**2 - alpha)) * dt
        
        # Apply boundaries
        outside_range = torch.abs(x) > 1.5
        x = torch.where(outside_range, 1.5 * torch.sign(x), x)
        
        # Keep error positive
        e = torch.clamp(e, min=0.01)
    
    # Get binary solution by taking the sign
    binary_solution = torch.sign(x).squeeze(1)
    
    # Calculate Ising energy: -0.5 * ∑∑J_ij*s_i*s_j - ∑h_i*s_i
    ising_energy = -0.5 * torch.sum(binary_solution @ J * binary_solution, dim=0) - torch.matmul(h.T, binary_solution)
    
    return binary_solution, ising_energy.squeeze()


In [21]:
def cac_function(J, h=None, steps=1000, agents=1, dt=0.075):
    """
    Run a Coherent Ising Machine with Chaotic Amplitude Control (CAC) simulation.
    
    Args:
        J: Coupling matrix defining the optimization problem
        h: External field vector (optional)
        steps: Number of simulation steps
        agents: Number of parallel optimization agents
        dt: Time step size
    
    Returns:
        binary_solution: The final binary solution (±1 values)
        ising_energy: The Ising energy of the final solution
    """
    # Convert inputs to float tensors
    J = J.float()
    if h is not None:
        h = h.float()
    else:
        h = torch.zeros(J.shape[0], device=J.device)
    
    # Get problem size
    N = J.shape[0]
    device = J.device
    
    # Initialize positions and error variables
    x = torch.randn(N, agents, device=device) * 1e-4  # Position variables
    e = torch.ones((N, agents), dtype=torch.float32, device=device)  # Error variables
    h = h.reshape(-1, 1)  # Make h a column vector
    
    # Define parameter schedules
    rise_steps = int(0.9 * steps)
    plateau_steps = steps - rise_steps
    
    # Create pump parameter schedule (p_t)
    p_rise = torch.linspace(-0.5, 1, rise_steps, device=device)
    p_plateau = torch.ones(plateau_steps, device=device)
    p_values = torch.cat([p_rise, p_plateau])
    
    # Create target amplitude schedule (alpha_t)
    alpha_rise = torch.linspace(1, 3, rise_steps, device=device)
    alpha_plateau = 3.0 * torch.ones(plateau_steps, device=device)
    alpha_values = torch.cat([alpha_rise, alpha_plateau])
    
    # Set fixed parameters
    beta = 0.3  # Error feedback gain
    
    # Calculate scaling factor for coupling
    xi = torch.sqrt(torch.tensor(2.0 * N / torch.sum(J**2), device=device))
    
    # Main simulation loop
    for step in range(steps):
        # Get current parameter values
        p_t = p_values[step]
        alpha_t = alpha_values[step]
        
        # Update positions (cubic nonlinearity, pump, feedback with error modulation)
        x = x + (-x**3 + (p_t - 1) * x + xi * e * (torch.matmul(J, x) + h)) * dt
        
        # Update error variables (amplitude-based control)
        e = e + (-beta * e * (x**2 - alpha_t)) * dt
        
        # Apply amplitude-dependent boundaries
        threshold = 1.5 * torch.sqrt(alpha_t)
        outside_range = torch.abs(x) > threshold
        x = torch.where(
            outside_range, 
            1.5 * torch.sign(x) * torch.sqrt(alpha_t), 
            x
        )
    
    # Get binary solution by taking the sign
    binary_solution = torch.sign(x).squeeze(1)
    
    # Calculate Ising energy: -0.5 * ∑∑J_ij*s_i*s_j - ∑h_i*s_i
    ising_energy = -0.5 * torch.sum(binary_solution @ J * binary_solution, dim=0) - torch.matmul(h.T, binary_solution)
    
    return binary_solution, ising_energy.squeeze()


In [22]:
def lqa_function(J, h=None, steps=1000, agents=1, c0=0.1, dt=1.0, beta1=0.9, beta2=0.999, epsilon=1e-8):
    """
    Run a Local Quantum Annealing (LQA) simulation.
    
    Args:
        J: Coupling matrix defining the optimization problem
        h: External field vector (optional)
        steps: Number of simulation steps
        agents: Number of parallel optimization agents
        c0: Coupling strength
        dt: Time step size
        beta1: Adam beta1 parameter
        beta2: Adam beta2 parameter
        epsilon: Adam epsilon parameter
    
    Returns:
        binary_solution: The final binary solution (±1 values)
        ising_energy: The Ising energy of the final solution
    """
    # Convert inputs to float tensors
    J = J.float()
    if h is not None:
        h = h.float()
    else:
        h = torch.zeros(J.shape[0], device=J.device)
    
    # Get problem size
    N = J.shape[0]
    device = J.device
    
    # Initialize position variables
    x = (torch.rand(N, agents, device=device) - 0.5) * 0.2  # Position variables
    h = h.reshape(-1, 1)  # Make h a column vector
    
    # Adam optimizer state
    m = torch.zeros_like(x)  # First moment
    v = torch.zeros_like(x)  # Second moment
    
    # Main simulation loop
    for step in range(1, steps):
        # Calculate annealing parameter (time-dependent)
        t = step / steps
        
        # Calculate quantum state vectors (based on position)
        theta = torch.pi / 2 * torch.tanh(x)
        z = torch.sin(theta)  # z-component (corresponds to classical spin)
        y_comp = torch.cos(theta)  # y-component (quantum component)
        
        # Calculate gradient
        # -t * c0 * (J@z + h) * y is the coupling term
        # (1 - t) * z is the transverse field term
        force = torch.pi / 2 * (-t * c0 * (torch.matmul(J, z) + h) * y_comp + (1 - t) * z) * (1 - torch.tanh(x) ** 2)
        
        # Adam update
        m = beta1 * m + (1 - beta1) * force
        v = beta2 * v + (1 - beta2) * force**2
        
        # Bias correction
        m_corrected = m / (1 - beta1 ** step)
        v_corrected = v / (1 - beta2 ** step)
        
        # Update position values
        x = x - dt * m_corrected / (torch.sqrt(v_corrected) + epsilon)
    
    # Get binary solution by taking the sign
    binary_solution = torch.sign(x).squeeze(1)
    
    # Calculate Ising energy: -0.5 * ∑∑J_ij*s_i*s_j - ∑h_i*s_i
    ising_energy = -0.5 * torch.sum(binary_solution @ J * binary_solution, dim=0) - torch.matmul(h.T, binary_solution)
    
    return binary_solution, ising_energy.squeeze()



In [23]:
def nmfa_function(J, h=None, steps=1000, agents=1, alpha=0.15, sigma=0.15):
    """
    Run a Noisy Mean-Field Annealing (NMFA) simulation.
    
    Args:
        J: Coupling matrix defining the optimization problem
        h: External field vector (optional)
        steps: Number of simulation steps
        agents: Number of parallel optimization agents
        alpha: Momentum factor
        sigma: Standard deviation of noise
    
    Returns:
        binary_solution: The final binary solution (±1 values)
        ising_energy: The Ising energy of the final solution
    """
    # Convert inputs to float tensors
    J = J.float()
    if h is not None:
        h = h.float()
    else:
        h = torch.zeros(J.shape[0], device=J.device)
    
    # Get problem size
    N = J.shape[0]
    device = J.device
    
    # Initialize position variables
    x = torch.zeros((N, agents), dtype=torch.float32, device=device)  # Position variables
    h = h.reshape(-1, 1)  # Make h a column vector
    
    # Calculate normalization factor for the fields
    J_squared = J**2
    J_squared_sum = J_squared.sum(dim=1, keepdim=True)
    h_squared_sum = (h**2).sum() if h is not None else 0
    J_norm = torch.sqrt(J_squared_sum + h_squared_sum)
    
    # Initialize inverse temperature (beta)
    beta = 1.0 / steps
    
    # Main simulation loop
    for step in range(steps):
        # Calculate mean field with noise
        # Normalize coupling to avoid exploding values
        mean_field = (torch.matmul(J, x) + h) / J_norm
        
        # Add noise (temperature-dependent)
        noise = torch.randn_like(x) * sigma
        field_with_noise = mean_field + noise
        
        # Apply tanh transformation with temperature
        x_new = torch.tanh(field_with_noise * beta)
        
        # Update with momentum
        x = alpha * x_new + (1 - alpha) * x
        
        # Increase inverse temperature (cooling schedule)
        beta += 1.0 / steps
    
    # Get binary solution by taking the sign
    binary_solution = torch.sign(x).squeeze(1)
    
    # Calculate Ising energy: -0.5 * ∑∑J_ij*s_i*s_j - ∑h_i*s_i
    ising_energy = -0.5 * torch.sum(binary_solution @ J * binary_solution, dim=0) - torch.matmul(h.T, binary_solution)
    
    return binary_solution, ising_energy.squeeze()


In [24]:
torch.manual_seed(42)



# Example usage
J = torch.tensor([[0, -2, 3], [-2, 0, 1], [3, 1, 0]])
h = torch.tensor([1, -4, 2])

# Run optimization
s, energy = simcim_function(J, h)
print("Final spins simcim:", s)
print("Energy simcim:", energy)

assert -11.0 == energy
assert s.tolist() in [[1, -1, 1], [-1, 1, -1]]

# Run optimization
s, energy = sfc_function(J, h)
print("Final spins sfc:", s)
print("Energy sfc:", energy)

assert -11.0 == energy
assert s.tolist() in [[1, -1, 1], [-1, 1, -1]]


# Run optimization
s, energy = cfc_function(J, h)
print("Final spins cfc:", s)
print("Energy cfc:", energy)

assert -11.0 == energy
assert s.tolist() in [[1, -1, 1], [-1, 1, -1]]

# Run optimization
s, energy = cac_function(J, h)
print("Final spins cac:", s)
print("Energy cac:", energy)

assert -11.0 == energy
assert s.tolist() in [[1, -1, 1], [-1, 1, -1]]

# Run optimization
s, energy = lqa_function(J, h)
print("Final spins lqa:", s)
print("Energy lqa:", energy)

assert -11.0 == energy
assert s.tolist() in [[1, -1, 1], [-1, 1, -1]]

# Run optimization
s, energy = nmfa_function(J, h)
print("Final spins nmfa:", s)
print("Energy nmfa:", energy)

assert -11.0 == energy
assert s.tolist() in [[1, -1, 1], [-1, 1, -1]]


Final spins simcim: tensor([ 1., -1.,  1.])
Energy simcim: tensor(-11.)
Final spins sfc: tensor([ 1., -1.,  1.])
Energy sfc: tensor(-11.)
Final spins cfc: tensor([ 1., -1.,  1.])
Energy cfc: tensor(-11.)
Final spins cac: tensor([ 1., -1.,  1.])
Energy cac: tensor(-11.)
Final spins lqa: tensor([ 1., -1.,  1.])
Energy lqa: tensor(-11.)
Final spins nmfa: tensor([ 1., -1.,  1.])
Energy nmfa: tensor(-11.)


/var/folders/tt/3h_jlt8571z664b95jr80v_40000gn/T/ipykernel_6233/2992097619.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xi = torch.sqrt(torch.tensor(2.0 * N / torch.sum(J**2), device=device))
/var/folders/tt/3h_jlt8571z664b95jr80v_40000gn/T/ipykernel_6233/1518615857.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xi = torch.sqrt(torch.tensor(2.0 * N / torch.sum(J**2), device=device))
/var/folders/tt/3h_jlt8571z664b95jr80v_40000gn/T/ipykernel_6233/3656101911.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xi = torch.sqrt(